# Init

In [16]:
from pandas_data_reader_service_core.modules.finam.search_service import Search, Market
from pandas_data_reader_service_core.modules.finam.stock_info import FinamStockInfo
from pandas_data_reader_service_core.service import PandasDataReaderService as pdrs, StockInfo, TimeFrame

from datetime import date

import plotly.graph_objects as go

import pandas as pd
from datetime import datetime
from typing import Tuple, List

In [17]:
arr = [pd.Timedelta(hours =1, minutes= 1), pd.Timedelta(hours =1, days=1 ), pd.Timedelta(hours =1)]
arr.sort()
arr

[Timedelta('0 days 01:00:00'),
 Timedelta('0 days 01:01:00'),
 Timedelta('1 days 01:00:00')]

In [18]:
stock = "EURUSD"
market = Market.CURRENCIES_WORLD

date_from = date(2020,4,6)
date_till = date(2021,7,14)
tf = TimeFrame.DAILY

# Define test tool

In [19]:
from src.ElliotWaves.misc.point import point
from __future__ import annotations


In [20]:
class candle_chart:
    def __init__(self,df: pd.DataFrame) -> None:
        self.fig = go.Figure(data=[go.Candlestick(x=df.index,
            open= df['O'],
            high= df['H'],
            low=  df['L'],
            close=df['C'])])
        self.fig.update_yaxes(fixedrange=False)
        self.fig.update_layout(height=1000)
        self.max = df.H.max()
        self.min = df.L.min()
        pass

    def add_marker(self, marker_candle:pd.Series)->candle_chart:
        self.fig.add_shape(type="line",
            x0=marker_candle.name, x1=marker_candle.name, y0=self.min, y1=marker_candle["L"],
            line=dict(
                color="MediumPurple",
                width=4,
                dash="dot",
            )
        )

        self.fig.add_shape(type="line",
            x0=marker_candle.name, x1=marker_candle.name, y0=marker_candle["H"], y1=self.max,
            line=dict(
                color="MediumPurple",
                width=4,
                dash="dot",
            )
        )
        return self
    
    def add_wave_from_point(self, start_point: point, end_point:point, main_wave: bool = True)->candle_chart:
        return self.add_wave(start_point.timestamp,start_point.price,end_point.timestamp,end_point.price,main_wave)
        
    def add_wave(self, start_dt:pd.Timestamp, start_value:float, end_dt:pd.Timestamp, end_value:float, main_wave: bool = True)->candle_chart:
        if end_value > start_value:
            color = "Green"
        else:
            color = "Red"
        if main_wave:
            dash = "solid"
        else:
            dash = "dashdot"
        self.fig.add_shape(type="line",
            x0=start_dt, x1=end_dt, y0=start_value, y1=end_value,
            line=dict(
                color=color,
                width=2,
                dash=dash,
            )
        )
        return self
    def show(self):
        self.fig.show()


# Search Test

## Define Test data

In [21]:
def grow(arr:List[int], cur:int, times)-> Tuple[List[int], int]:
    for wave in range(times):
        arr.append(cur)
        cur = cur + 1
    return arr, cur

def fall(arr:List[int], cur:int, times)-> Tuple[List[int], int]:
    for wave in range(times):
        arr.append(cur)
        cur = cur - 1
    return arr, cur
    
high_arr,cur = grow([], 1, 3)
high_arr,cur = fall(high_arr, cur, 2)
high_arr,cur = grow(high_arr, cur, 7)
high_arr,cur = fall(high_arr, cur, 3)
high_arr,cur = grow(high_arr, cur, 10)

times = []
for day in range(len(high_arr)):
    times.append(pd.Timestamp(year=2021, month=1, day=day+1))

low_arr = [h-1 for h in high_arr]

mid_arr = []
for idx in range(len(low_arr)):
    mid_arr.append((high_arr[idx]+low_arr[idx])/2)

df = pd.DataFrame(data={
    "O": mid_arr,
    "C": mid_arr,
    "H": high_arr,
    "L": low_arr},
    index=times)
candle_chart(df).show()

## Search test

In [22]:
from src.ElliotWaves.misc.direction import direction
from src.ElliotWaves.waves.impulse_five_search import search
from src.ElliotWaves.misc.wave import wave
from src.ElliotWaves.misc.point import point

In [23]:
waves, dead = search(df,impulse_direction=direction.Long)
print(len(waves))

1


In [24]:
chrt = candle_chart(df)
sb_wv = waves[0].sub_waves
for idx in range(5):
    wv:wave = sb_wv[idx]
    chrt.add_wave(wv.start.timestamp, wv.start.price, wv.end.timestamp,wv.end.price)
chrt.show()

## Define data for Short test

In [25]:
cur = 15
high_arr,cur = fall([], cur, 9)
high_arr,cur = grow(high_arr, cur, 3)
high_arr,cur = fall(high_arr, cur, 7)
high_arr,cur = grow(high_arr, cur, 2)
high_arr,cur = fall(high_arr, cur, 4)
        
# № | idx   |  ts   | pr H | pr L | Type  | W pr |
#---|-------|-------|------|------|-------|------|
# 0 | 00-09 | 01-10 | 15-6 | 14-5 | Short | 15-5 |
# 1 | 09-12 | 10-13 | 6-9  | 5-8  | Long  | 5-9  |
# 2 | 12-19 | 13-20 | 9-2  | 8-1  | Short | 9-1  |
# 3 | 19-21 | 20-22 | 2-4  | 1-3  | Long  | 1-4  |
# 4 | 21-24 | 22-25 | 4-1  | 3-0  | Short | 4-0  |

times = []
for day in range(len(high_arr)):
    times.append(pd.Timestamp(year=2021, month=1, day=day+1))

low_arr = [h-1 for h in high_arr]
mid_arr = []
for idx in range(len(low_arr)):
    mid_arr.append((high_arr[idx]+low_arr[idx])/2)
df_sh = pd.DataFrame(data={
    "O": mid_arr,
    "C":mid_arr,
    "H": high_arr,
    "L": low_arr},
    index=times)
candle_chart(df_sh).show()

## Search short test

In [26]:
waves,dead = search(df_sh,impulse_direction=direction.Short)
len(waves)

1

In [27]:
chrt = candle_chart(df_sh)
sb_wv = waves[0].sub_waves
for idx in range(5):
    wv:wave = sb_wv[idx]
    chrt.add_wave(wv.start.timestamp, wv.start.price, wv.end.timestamp,wv.end.price)
chrt.show()

## Test search

In [28]:
waves,dead = search(df,impulse_direction=direction.Short)
len(waves)

0

# Test dead wave

In [29]:
high_arr,cur = grow([], 1, 3)
high_arr,cur = fall(high_arr, cur, 2)
high_arr,cur = grow(high_arr, cur, 7)
high_arr,cur = fall(high_arr, cur, 3)
high_arr,cur = grow(high_arr, cur, 3)

times = []
for day in range(len(high_arr)):
    times.append(pd.Timestamp(year=2021, month=1, day=day+1))

low_arr = [h-1 for h in high_arr]

mid_arr = []
for idx in range(len(low_arr)):
    mid_arr.append((high_arr[idx]+low_arr[idx])/2)

df = pd.DataFrame(data={
    "O": mid_arr,
    "C": mid_arr,
    "H": high_arr,
    "L": low_arr},
    index=times)
candle_chart(df).show()

In [30]:
waves, deads = search(df,impulse_direction=direction.Long)
print(f"waves = {len(waves)}, dead = {len(deads)}")

waves = 0, dead = 6


In [31]:
deads_sub_waves = []
for dead in deads:
    sb_wv = dead.sub_waves
    if len(sb_wv) == 0:
        deads_sub_waves.append([dead])
    else:
        deads_sub_waves.append(sb_wv)
for sb_wv in deads_sub_waves:
    print(len(sb_wv)) 

1
2
1
2
3
4


In [32]:
for sb_wv in deads_sub_waves:
    chrt = candle_chart(df)
    for idx in range(len(sb_wv)):
        wv:wave = sb_wv[idx]
        chrt.add_wave(wv.start.timestamp, wv.start.price, wv.end.timestamp,wv.end.price)
    chrt.show()